<a href="https://colab.research.google.com/github/Jurgo001/TH_TriTueNhanTao/blob/main/Buoi01_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import copy
from heapq import heappush, heappop
import math
import random

# --- I. THÔNG SỐ CƠ BẢN (GLOBAL/TĨNH) ---

O_TRONG = 0 # Giá trị đại diện cho ô trống (Blank Tile)

# 4 hướng dịch chuyển: (Hàng, Cột)
DOI_HANG = [ 1, 0, -1, 0 ] # [Xuống, Giữ nguyên, Lên, Giữ nguyên]
DOI_COT = [ 0, -1, 0, 1 ]  # [Giữ nguyên, Trái, Giữ nguyên, Phải]

# --- II. CÁC CẤU TRÚC DỮ LIỆU (OOP) ---

class HangDoiUuTien:
    """Lớp hàng đợi ưu tiên (Min-Heap) - Quản lý Open Set."""

    def __init__(self):
        # Khởi tạo heap rỗng. Heap sẽ chứa các Node chờ mở rộng.
        self.heap = []

    def day(self, khoa):
        """Đẩy Node mới vào heap. Thư viện heapq tự động sắp xếp theo F(n)."""
        heappush(self.heap, khoa)

    def lay(self):
        """Lấy Node có chi phí F(n) nhỏ nhất (Thực hiện Bước 2 AKT)."""
        return heappop(self.heap)

    def trong_khong(self):
        """Kiểm tra Open Set có rỗng không."""
        return not self.heap

class Nut:
    """Cấu trúc đại diện cho một trạng thái (node) của bàn cờ."""

    def __init__(self, cha, ma_tran, vi_tri_o_trong, h_chi_phi, g_buoc_di):

        # G(n): Chi phí thực tế (Số bước đã đi)
        self.g_buoc_di = g_buoc_di

        # H(n): Chi phí Heuristic (Số ô sai vị trí)
        self.h_chi_phi = h_chi_phi

        # F(n): Tổng chi phí A* (F(n) = G(n) + H(n)) - Tính toán chi phí đánh giá.
        self.f_cost = self.g_buoc_di + self.h_chi_phi

        # self.parent: Con trỏ truy vết ngược về Node cha.
        self.parent = cha

        # self.mats: Ma trận trạng thái hiện tại.
        self.mats = ma_tran

        # self.empty_tile_posi: Vị trí [hàng, cột] của ô trống.
        self.empty_tile_posi = vi_tri_o_trong

    def __lt__(self, nxt):
        """Hàm so sánh (Comparison) - Cốt lõi của thuật toán A KT."""
        # Tác dụng Code: Node có F(n) nhỏ hơn sẽ được ưu tiên mở rộng.
        return self.f_cost < nxt.f_cost

# --- III. CÁC HÀM LOGIC VÀ HEURISTIC ---

def tinh_chi_phi_h(mats, final, N) -> int:
    """Tính H(n) - Heuristic Số ô sai vị trí (Misplaced Tile)."""
    # Note: Hàm này nhận N động để tính cho mọi kích thước N x N.
    count = 0
    for i in range(N):
        for j in range(N):
            # Nếu ô không phải là ô trống VÀ không ở đúng vị trí đích
            if mats[i][j] != O_TRONG and mats[i][j] != final[i][j]:
                count += 1
    return count

def tao_nut_moi(mats, empty_tile_posi, new_empty_tile_posi,
             levels_cha, parent, final, N) -> Nut:
    """Tạo Node con mới sau khi dịch chuyển ô trống (Bước 3 AKT)."""

    new_mats = copy.deepcopy(mats)
    x1, y1 = empty_tile_posi[0], empty_tile_posi[1]
    x2, y2 = new_empty_tile_posi[0], new_empty_tile_posi[1]

    # Hoán đổi ô trống và ô lân cận
    new_mats[x1][y1], new_mats[x2][y2] = new_mats[x2][y2], new_mats[x1][y1]

    # Tính G(S) mới: G(cha) + 1
    g_moi = levels_cha + 1
    # Tính H(S) mới
    h_moi = tinh_chi_phi_h(new_mats, final, N)

    # Tạo Node con với F(n) = G(n) + H(n) được tính toán.
    child = Nut(parent, new_mats, new_empty_tile_posi, h_moi, g_moi)
    return child

def in_ma_tran(mats, N):
    """In ma trận N x N (Note: Sử dụng f-string formatting để căn chỉnh)."""
    for i in range(N):
        print(" ".join(f"{mats[i][j]:2}" for j in range(N)))

def la_an_toan(x, y, N):
    """Kiểm tra xem tọa độ [x, y] có nằm trong biên N x N không."""
    return 0 <= x < N and 0 <= y < N

def in_duong_di(root, N):
    """In đường đi giải pháp (Sử dụng đệ quy truy vết ngược qua self.parent)."""
    if root is None:
        return
    in_duong_di(root.parent, N)
    # Hiển thị chi phí F(n) chi tiết tại mỗi bước giải.
    print(f"Bước (G(n)): {root.g_buoc_di}")
    print(f"Chi phí F(n) = {root.f_cost} (G={root.g_buoc_di} + H={root.h_chi_phi})")
    in_ma_tran(root.mats, N)
    print()

def tao_trang_thai_dich_chuan(N):
    """Tạo ma trận trạng thái đích chuẩn N x N (Target State)."""
    # Note: Ô trống 0 luôn ở góc dưới bên phải.
    ma_tran_dich = []
    gia_tri = 1
    for i in range(N):
        hang = []
        for j in range(N):
            hang.append(gia_tri)
            gia_tri += 1
        ma_tran_dich.append(hang)
    ma_tran_dich[N-1][N-1] = O_TRONG
    return ma_tran_dich

def tinh_so_nghich_the(ma_tran_phang):
    """Tính số nghịch thế (Inversion Count) của ma trận phẳng."""
    nghich_the = 0
    # Note: Cần thiết để kiểm tra tính khả giải của N-Puzzle.
    for i in range(len(ma_tran_phang)):
        for j in range(i + 1, len(ma_tran_phang)):
            if ma_tran_phang[i] > 0 and ma_tran_phang[j] > 0 and ma_tran_phang[i] > ma_tran_phang[j]:
                nghich_the += 1
    return nghich_the

def kiem_tra_kha_giai(trang_thai_dau, vi_tri_o_trong_dau, N) -> bool:
    """Kiểm tra tính khả giải theo quy tắc Inversion Count."""
    # Tác dụng Code: Chặn trạng thái không thể giải được (tối ưu hóa Demo).
    ma_tran_phang = [tile for hang in trang_thai_dau for tile in hang if tile != O_TRONG]
    so_nghich_the = tinh_so_nghich_the(ma_tran_phang)
    hang_o_trong = vi_tri_o_trong_dau[0] + 1

    if N % 2 != 0: # N lẻ (3x3, 5x5)
        return so_nghich_the % 2 == 0 # Khả giải nếu số nghịch thế CHẴN
    else: # N chẵn (4x4)
        # Khả giải nếu (Số nghịch thế % 2) == (Hàng ô trống % 2)
        return (so_nghich_the % 2) == (hang_o_trong % 2)

# --- IV. HÀM TẠO TRẠNG THÁI NGẪU NHIÊN KHẢ GIẢI ---

def hoan_doi_ngau_nhien(mats, vi_tri_o_trong, N):
    """Hoán đổi ô trống với một ô lân cận an toàn."""
    x, y = vi_tri_o_trong[0], vi_tri_o_trong[1]

    cac_huong_an_toan = []
    for i in range(4):
        x_moi = x + DOI_HANG[i]
        y_moi = y + DOI_COT[i]
        if la_an_toan(x_moi, y_moi, N):
            cac_huong_an_toan.append((x_moi, y_moi))

    if not cac_huong_an_toan:
        return mats, vi_tri_o_trong

    x_hoan_doi, y_hoan_doi = random.choice(cac_huong_an_toan)

    mats[x][y], mats[x_hoan_doi][y_hoan_doi] = mats[x_hoan_doi][y_hoan_doi], mats[x][y]

    return mats, [x_hoan_doi, y_hoan_doi]

def tao_trang_thai_ngau_nhien_kha_giai(N, so_buoc_xao_tron) -> tuple:
    """Tạo trạng thái đầu khả giải bằng cách xáo trộn trạng thái đích."""
    # Tác dụng Code: Bắt đầu từ đích và xáo trộn N lần để đảm bảo tính khả giải.

    initial = tao_trang_thai_dich_chuan(N)

    vi_tri_o_trong = None
    for i in range(N):
        for j in range(N):
            if initial[i][j] == O_TRONG:
                vi_tri_o_trong = [i, j]
                break
        if vi_tri_o_trong: break

    for _ in range(so_buoc_xao_tron):
        initial, vi_tri_o_trong = hoan_doi_ngau_nhien(initial, vi_tri_o_trong, N)

    return initial, vi_tri_o_trong

# --- V. THUẬT TOÁN A KT (A-star) ---

def chuyen_ma_tran_sang_tuple(mats):
    """Chuyển ma trận 2D sang tuple phẳng (hashable) để lưu vào Closed Set."""
    return tuple(tile for hang in mats for tile in hang)

def giai_puzzle_akt(initial, empty_tile_posi, final, N):
    """Hàm chính giải N-Puzzle bằng thuật toán A KT (A-star)."""

    if not kiem_tra_kha_giai(initial, empty_tile_posi, N):
        print("\n!!! LỖI QUAN TRỌNG: MA TRẬN KHÔNG KHẢ GIẢI. Không thể tìm thấy lời giải !!!")
        return

    pq = HangDoiUuTien()
    # Dữ liệu Động 6: Closed Set (TrangThai_DaTham) - Tối ưu hóa không gian tìm kiếm.
    TrangThai_DaTham = set()

    h_goc = tinh_chi_phi_h(initial, final, N)
    root = Nut(None, initial, empty_tile_posi, h_goc, 0) # Bước 1: Khởi tạo Node gốc G(n)=0
    pq.day(root)

    # 2. Vòng lặp tìm kiếm chính (Bước 4: Quay lại Bước 2)
    while not pq.trong_khong():

        # Bước 2: Chọn đỉnh N có F(n) min
        minimum = pq.lay()

        # Bước 3: Đóng đỉnh N
        TrangThai_DaTham.add(chuyen_ma_tran_sang_tuple(minimum.mats))

        # Kiểm tra điều kiện đích
        if minimum.h_chi_phi == 0:
            print("--- ĐÃ TÌM THẤY GIẢI PHÁP ---")
            in_duong_di(minimum, N)
            return

        # 3. Mở mọi đỉnh sau N (Phát triển Node con)
        for i in range(4):
            new_tile_posi = [
                minimum.empty_tile_posi[0] + DOI_HANG[i],
                minimum.empty_tile_posi[1] + DOI_COT[i], ]

            x_moi, y_moi = new_tile_posi[0], new_tile_posi[1]

            if la_an_toan(x_moi, y_moi, N):

                child = tao_nut_moi(minimum.mats,
                                 minimum.empty_tile_posi,
                                 new_tile_posi,
                                 minimum.g_buoc_di,
                                 minimum, final, N)

                # Logic tối ưu: Chỉ thêm Node con nếu trạng thái chưa từng được duyệt.
                if chuyen_ma_tran_sang_tuple(child.mats) not in TrangThai_DaTham:
                    pq.day(child)

    print("KHÔNG TÌM THẤY GIẢI PHÁP (Hết không gian tìm kiếm).")


# --- VI. THỰC THI CHÍNH (TÍCH HỢP CHẾ ĐỘ CHỌN INPUT) ---

def lay_trang_thai_nhap_tay(N):
    """Xử lý logic nhập thủ công trạng thái ban đầu."""
    ma_tran_dau = []
    vi_tri_o_trong_dau = None

    print(f"\n--- NHẬP THỦ CÔNG {N}x{N} ---")
    print(f"Nhập {N*N} số (0 là ô trống) cách nhau bằng dấu cách (VD khả giải: 1 2 3 4 5 6 7 8 0):")

    while True:
        try:
            # Dữ liệu Động 7: Dữ liệu nhập thô từ người dùng.
            du_lieu_nhap = list(map(int, input("Nhập dữ liệu: ").split()))

            # Kiểm tra lỗi số lượng và giá trị (Code kiểm tra lỗi đầu vào)
            if len(du_lieu_nhap) != N * N or set(du_lieu_nhap) != set(range(N * N)):
                print(f"Lỗi: Cần nhập đúng {N*N} số duy nhất trong khoảng 0 đến {N*N - 1}.")
                continue

            # Xử lý và trả về ma trận, vị trí ô trống.
            k = 0
            for i in range(N):
                hang = du_lieu_nhap[k : k + N]
                ma_tran_dau.append(hang)
                for j in range(N):
                    if ma_tran_dau[i][j] == O_TRONG:
                        vi_tri_o_trong_dau = [i, j]
                k += N
            return ma_tran_dau, vi_tri_o_trong_dau

        except ValueError:
            print("Lỗi: Vui lòng chỉ nhập các số nguyên.")
            return None, None
        except Exception:
            return None, None

def thuc_thi_chinh():

    # 1. Nhập kích thước N
    # Dữ liệu Động 8: Kích thước N (Dynamic Input chính).
    try:
        N = int(input("Nhập kích thước N của N-Puzzle (Ví dụ: 3): "))
    except ValueError:
        N = 3

    if N < 2:
        print("Kích thước phải >= 2.")
        return

    # 2. Chọn chế độ Input
    print("\n--- CHỌN CHẾ ĐỘ NHẬP DỮ LIỆU ---")
    print("1. Tự nhập trạng thái ban đầu.")
    print("2. Tự động xáo trộn từ trạng thái đích.")

    che_do = input("Chọn chế độ (1 hoặc 2): ")

    # Tạo trạng thái đích trước
    # Dữ liệu Động 10: Trạng thái đích được tạo động dựa trên N.
    trang_thai_dich = tao_trang_thai_dich_chuan(N)

    ma_tran_dau = []
    vi_tri_o_trong_dau = None
    so_buoc_xao_tron = 0

    if che_do == '1':
        ma_tran_dau, vi_tri_o_trong_dau = lay_trang_thai_nhap_tay(N)
        if ma_tran_dau is None:
            return

    elif che_do == '2':
        # Dữ liệu Động 9: Nhập số bước xáo trộn (kiểm soát độ khó).
        try:
            so_buoc_xao_tron = int(input(f"Nhập số bước xáo trộn (độ khó) so với đích (VD: 10, 20): "))
        except ValueError:
            so_buoc_xao_tron = 10

        print(f"\n--- TỰ ĐỘNG XÁO TRỘN {N}x{N} với {so_buoc_xao_tron} bước ---")
        # Gọi hàm tạo trạng thái ngẫu nhiên khả giải
        ma_tran_dau, vi_tri_o_trong_dau = tao_trang_thai_ngau_nhien_kha_giai(N, so_buoc_xao_tron)

    else:
        print("Chế độ chọn không hợp lệ. Đang thoát.")
        return

    # 3. Hiển thị và Giải
    print("\n" + "="*50)
    print("--- BẮT ĐẦU GIẢI N-PUZZLE BẰNG THUẬT TOÁN A KT ---")
    print(f"Kích thước bàn cờ: {N}x{N}")
    if che_do == '2':
        print(f"Độ khó dự kiến: ~{so_buoc_xao_tron} bước")
    print("Trạng thái đầu:")
    in_ma_tran(ma_tran_dau, N)
    print("\nTrạng thái đích:")
    in_ma_tran(trang_thai_dich, N)
    print("Lưu ý: Với N > 3, thuật toán AKT sẽ RẤT CHẬM.")
    print("="*50)

    # Gọi phương thức giải A KT (Truyền N động)
    giai_puzzle_akt(ma_tran_dau, vi_tri_o_trong_dau, trang_thai_dich, N)

thuc_thi_chinh()

Nhập kích thước N của N-Puzzle (Ví dụ: 3): 3

--- CHỌN CHẾ ĐỘ NHẬP DỮ LIỆU ---
1. Tự nhập trạng thái ban đầu.
2. Tự động xáo trộn từ trạng thái đích.
Chọn chế độ (1 hoặc 2): 2
Nhập số bước xáo trộn (độ khó) so với đích (VD: 10, 20): 30

--- TỰ ĐỘNG XÁO TRỘN 3x3 với 30 bước ---

--- BẮT ĐẦU GIẢI N-PUZZLE BẰNG THUẬT TOÁN A KT ---
Kích thước bàn cờ: 3x3
Độ khó dự kiến: ~30 bước
Trạng thái đầu:
 2  7  5
 1  0  3
 4  8  6

Trạng thái đích:
 1  2  3
 4  5  6
 7  8  0
Lưu ý: Với N > 3, thuật toán AKT sẽ RẤT CHẬM.
--- ĐÃ TÌM THẤY GIẢI PHÁP ---
Bước (G(n)): 0
Chi phí F(n) = 7 (G=0 + H=7)
 2  7  5
 1  0  3
 4  8  6

Bước (G(n)): 1
Chi phí F(n) = 8 (G=1 + H=7)
 2  0  5
 1  7  3
 4  8  6

Bước (G(n)): 2
Chi phí F(n) = 9 (G=2 + H=7)
 2  5  0
 1  7  3
 4  8  6

Bước (G(n)): 3
Chi phí F(n) = 9 (G=3 + H=6)
 2  5  3
 1  7  0
 4  8  6

Bước (G(n)): 4
Chi phí F(n) = 9 (G=4 + H=5)
 2  5  3
 1  7  6
 4  8  0

Bước (G(n)): 5
Chi phí F(n) = 11 (G=5 + H=6)
 2  5  3
 1  7  6
 4  0  8

Bước (G(n)): 6
Chi phí F(

In [ ]:
import math
from heapq import heappush, heappop # Sử dụng Min-Heap (heapq) để tối ưu O(log n)

# --- I. LỚP ĐỒ THỊ VÀ HÀM CỐT LÕI (OOP) ---

class DoThi:
    """Định nghĩa cấu trúc đồ thị và logic truy cập."""
    def __init__(self, danh_sach_ke):
        # Dữ liệu Động 1: Cấu trúc Đồ thị (Được nhập từ người dùng hoặc lấy từ mẫu)
        self.danh_sach_ke = danh_sach_ke

    def lay_dinh_ke(self, dinh_hien_tai):
        """Trả về [(Đỉnh kề, Trọng số), ...]."""
        return self.danh_sach_ke.get(dinh_hien_tai, [])

    def thuat_toan_a_sao(self, bat_dau, ket_thuc, ham_uoc_luong_h):
        """
        Tìm kiếm A* tối ưu. Công thức: f(n) = g(n) + h(n).
        Note: Đây là trung tâm của thuật toán, nơi dữ liệu động được xử lý.
        """
        # Dữ liệu Động 2: tap_mo_heap (Min-Heap/Hàng đợi Ưu tiên)
        # Tác dụng Code: Luôn giữ đỉnh có f(n) nhỏ nhất ở trên cùng (Tối ưu O(log n) cho việc tìm kiếm).
        tap_mo_heap = [(0 + ham_uoc_luong_h(bat_dau), bat_dau)]

        # Dữ liệu Động 3: tap_dong (Closed Set)
        # Tác dụng Code: Ghi nhớ các đỉnh đã được xử lý xong để tránh lặp lại.
        tap_dong = set()

        # Dữ liệu Động 4: chi_phi_g
        # Tác dụng Code: Lưu g(n) - Chi phí thực tế tối thiểu từ bắt đầu đến đỉnh n. Cập nhật khi tìm thấy đường ngắn hơn.
        chi_phi_g = {dinh: math.inf for dinh in self.danh_sach_ke}
        chi_phi_g[bat_dau] = 0

        # Dữ liệu Động 5: cha
        # Tác dụng Code: Lưu đỉnh cha (parent) để truy vết đường đi tối ưu khi tìm thấy đích.
        cha = {}

        while tap_mo_heap:
            # Tác dụng Code: Lấy đỉnh có f(n) thấp nhất ra khỏi heap.
            f_hien_tai, dinh_hien_tai = heappop(tap_mo_heap)

            if dinh_hien_tai == ket_thuc:
                # Tác dụng Code: Truy vết ngược và in ra đường đi tối ưu.
                duong_di = [ket_thuc]
                current = ket_thuc
                while current in cha:
                    current = cha[current]
                    duong_di.append(current)
                duong_di.reverse()

                print(f'Chi phí thực tế: {chi_phi_g[ket_thuc]}')
                return " -> ".join(duong_di)

            if dinh_hien_tai in tap_dong:
                continue

            # Tác dụng Code: Đưa đỉnh đã xử lý vào tập đóng.
            tap_dong.add(dinh_hien_tai)

            for (dinh_ke, trong_so) in self.lay_dinh_ke(dinh_hien_tai):
                # Tác dụng Code: Tính chi phí g(n) mới khi đi qua dinh_hien_tai.
                chi_phi_g_moi = chi_phi_g[dinh_hien_tai] + trong_so

                # Kiểm tra xem đường đi qua dinh_hien_tai có ngắn hơn đường đi đã biết không
                if chi_phi_g_moi < chi_phi_g.get(dinh_ke, math.inf):

                    # Tác dụng Code: Cập nhật đường đi ngắn hơn (g(n) và cha)
                    chi_phi_g[dinh_ke] = chi_phi_g_moi
                    cha[dinh_ke] = dinh_hien_tai

                    # Dữ liệu Động 6: f_moi
                    # Tác dụng Code: Tính f(n) mới và đẩy vào heap để xếp hàng ưu tiên.
                    f_moi = chi_phi_g_moi + ham_uoc_luong_h(dinh_ke)
                    heappush(tap_mo_heap, (f_moi, dinh_ke))

        return "Không tìm thấy đường đi!"

# --- II. HÀM INPUT ĐỘNG VÀ CHẠY MẪU (DEMO) ---

def lay_du_lieu_mau():
    """Tạo dữ liệu Đồ thị mẫu sẵn có cho chế độ chạy nhanh."""
    print("--- CHẠY CHẾ ĐỘ DEMO MẪU ---")

    ds_ke_mau = {
        'A': [('B', 6), ('F', 3)], 'B': [('C', 3), ('D', 2)],
        'C': [('D', 1), ('E', 5)], 'D': [('B', 2), ('E', 8)],
        'E': [('Z', 5)], 'F': [('G', 1)], 'G': [('I', 3)],
        'H': [('I', 2)], 'I': [('Z', 3)], 'Z': []
    }
    heuristic_mau = {
        'A': 10, 'B': 8, 'C': 5, 'D': 4, 'E': 3,
        'F': 10, 'G': 6, 'H': 5, 'I': 2, 'Z': 0
    }

    # Dữ liệu Động 7: Hàm Heuristic (Đóng gói Heuristic mẫu)
    def ham_uoc_luong_h_mau(n):
        return heuristic_mau.get(n, math.inf)

    return ds_ke_mau, ham_uoc_luong_h_mau, 'A', 'Z'

def nhap_du_lieu_do_thi():
    """Lấy đồ thị và heuristic từ người dùng (Dynamic Input)."""
    # Note: Đây là khối Dynamic Input quan trọng, cho phép thầy thay đổi các thông số.
    print("--- 1. NHẬP CẤU TRÚC ĐỒ THỊ (DYNAMIC INPUT) ---")

    ds_ke = {}
    while True:
        line = input("Cạnh (> END để kết thúc): ").upper()
        if line == 'END': break
        # Tác dụng Code: Phân tích cú pháp input của người dùng và xây dựng ds_ke.
        try:
            goc, dich, trong_so_str = line.split()
            trong_so = int(trong_so_str)
            if goc not in ds_ke: ds_ke[goc] = []
            if dich not in ds_ke: ds_ke[dich] = []
            ds_ke[goc].append((dich, trong_so))
        except:
            print("Lỗi. Nhập lại 'A B 6' (Ví dụ: Đỉnh_Gốc Đỉnh_Đích Trọng_số).")

    tat_ca_dinh = list(ds_ke.keys())

    print("\n--- 2. NHẬP HEURISTIC h(n) (DYNAMIC INPUT) ---")
    heuristic = {}
    for dinh in tat_ca_dinh:
        while True:
            try:
                chi_phi_str = input(f"h({dinh}) > ")
                # Dữ liệu Động 8: Giá trị Heuristic (Được gán trực tiếp bởi người dùng)
                heuristic[dinh] = int(chi_phi_str)
                break
            except ValueError:
                print("Lỗi: Vui lòng nhập số nguyên.")

    # Dữ liệu Động 7 (Hàm Heuristic - Tái định nghĩa cho input người dùng)
    def ham_uoc_luong_h(n):
        return heuristic.get(n, math.inf)

    return ds_ke, ham_uoc_luong_h, None, None

def thuc_thi_chinh():
    """Hàm Main để Demo."""
    # ... [Logic lựa chọn chế độ M/N] ...
    print("="*50)
    print("--- THUẬT TOÁN A* TÌM ĐƯỜNG ĐI NGẮN NHẤT ---")
    print("DEMO: Nhập 'M' (Mẫu) để chạy nhanh hoặc 'N' (Nhập) để nhập Dynamic Input.")
    print("="*50)

    lua_chon = input("Chọn chế độ (M/N): ").upper()

    if lua_chon == 'M':
        ds_ke, ham_h_input, bat_dau, ket_thuc = lay_du_lieu_mau()
    else:
        ds_ke, ham_h_input, _, _ = nhap_du_lieu_do_thi()

        if not ds_ke: return
        bat_dau = input("\nNhập Đỉnh Bắt đầu: ").upper()
        ket_thuc = input("Nhập Đỉnh Kết thúc: ").upper()

    if bat_dau not in ds_ke or ket_thuc not in ds_ke: return

    do_thi = DoThi(ds_ke)

    # Tác dụng Code: Hiển thị trạng thái giải để dễ dàng đối chiếu với thầy
    print("\n" + "="*50)
    print("--- BẮT ĐẦU GIẢI ---")
    if lua_chon == 'M':
         print("Đồ thị đang giải:")
         for k, v in ds_ke.items():
             print(f"  {k}: {v} (h={ham_h_input(k)})")
         print("-" * 20)

    ket_qua = do_thi.thuat_toan_a_sao(bat_dau, ket_thuc, ham_h_input)
    print(f"Đường đi tìm thấy: {ket_qua}")
    print("="*50)

thuc_thi_chinh()

--- THUẬT TOÁN A* TÌM ĐƯỜNG ĐI NGẮN NHẤT ---
DEMO: Nhập 'M' (Mẫu) để chạy nhanh hoặc 'N' (Nhập) để nhập Dynamic Input.
Chọn chế độ (M/N): m
--- CHẠY CHẾ ĐỘ DEMO MẪU ---

--- BẮT ĐẦU GIẢI ---
Đồ thị đang giải:
  A: [('B', 6), ('F', 3)] (h=10)
  B: [('C', 3), ('D', 2)] (h=8)
  C: [('D', 1), ('E', 5)] (h=5)
  D: [('B', 2), ('E', 8)] (h=4)
  E: [('Z', 5)] (h=3)
  F: [('G', 1)] (h=10)
  G: [('I', 3)] (h=6)
  H: [('I', 2)] (h=5)
  I: [('Z', 3)] (h=2)
  Z: [] (h=0)
--------------------
Chi phí thực tế: 10
Đường đi tìm thấy: A -> F -> G -> I -> Z
